In [ ]:
!pip install -q biopython

%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 2.3MB 19.0MB/s 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = '/content/drive/MyDrive/data/chip-seq/'
LOG_DIR = '/content/drive/MyDrive/dna-nn/results/'

In [ ]:
!cp -r '/content/drive/MyDrive/dna-nn/dna_nn' .
!ls dna_nn

dataset.py  download.py  load.py  model.py


In [ ]:
import gc

import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras

from dna_nn.dataset import motif_discovery
from dna_nn.model import models, evaluate

In [ ]:
x_shape, train_ds, validation_data, test_ds = motif_discovery(DATA_DIR + 'motif_discovery_train.fasta', 
                                                              DATA_DIR + 'motif_discovery_test.fasta')

In [ ]:
models

{'cnn_deepdbp': <function dna_nn.model.cnn_deepdbp>,
 'cnn_nguyen_2_conv2d': <function dna_nn.model.cnn_nguyen_2_conv2d>,
 'cnn_nguyen_conv1d_2_conv2d': <function dna_nn.model.cnn_nguyen_conv1d_2_conv2d>,
 'cnn_zeng_2_conv2d': <function dna_nn.model.cnn_zeng_2_conv2d>,
 'cnn_zeng_3_conv2d': <function dna_nn.model.cnn_zeng_3_conv2d>,
 'cnn_zeng_4_conv2d': <function dna_nn.model.cnn_zeng_4_conv2d>}

In [ ]:
key = 'cnn_zeng_4_conv2d'
dataset = 'motif_discovery'

keras.backend.clear_session()
model = models[key](x_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 97, 190, 16)       160       
_________________________________________________________________
batch_normalization (BatchNo (None, 97, 190, 16)       64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 95, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 93, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 93, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 46, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 44, 64)        1

In [ ]:
csv_path = LOG_DIR + f'{key}-{dataset}-dynamics.csv'
model_path = LOG_DIR + f'{key}-{dataset}.h5'

callbacks = [
    keras.callbacks.CSVLogger(csv_path),
    keras.callbacks.LambdaCallback(
        on_epoch_end=lambda epoch, logs: gc.collect(),
        # on_train_end=lambda logs: model.save(model_path)
    ),
    keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)
]

history = model.fit(train_ds, epochs=50, validation_data=validation_data,
                    callbacks=callbacks, verbose=2)

Epoch 1/50
257/257 - 70s - loss: 0.7162 - accuracy: 0.5202 - val_loss: 0.7059 - val_accuracy: 0.5059
Epoch 2/50
257/257 - 70s - loss: 0.6802 - accuracy: 0.5557 - val_loss: 0.8291 - val_accuracy: 0.5059
Epoch 3/50
257/257 - 70s - loss: 0.6629 - accuracy: 0.5936 - val_loss: 0.6569 - val_accuracy: 0.6132
Epoch 4/50
257/257 - 69s - loss: 0.6492 - accuracy: 0.6192 - val_loss: 0.6601 - val_accuracy: 0.5820
Epoch 5/50
257/257 - 70s - loss: 0.6360 - accuracy: 0.6392 - val_loss: 0.6463 - val_accuracy: 0.6200
Epoch 6/50
257/257 - 70s - loss: 0.6218 - accuracy: 0.6552 - val_loss: 0.7765 - val_accuracy: 0.5330
Epoch 7/50
257/257 - 70s - loss: 0.6076 - accuracy: 0.6711 - val_loss: 0.7089 - val_accuracy: 0.5703
Epoch 8/50
257/257 - 70s - loss: 0.5935 - accuracy: 0.6848 - val_loss: 0.8451 - val_accuracy: 0.5257
Epoch 9/50
257/257 - 70s - loss: 0.5813 - accuracy: 0.6948 - val_loss: 0.6001 - val_accuracy: 0.6790
Epoch 10/50
257/257 - 69s - loss: 0.5662 - accuracy: 0.7080 - val_loss: 0.6734 - val_accura

In [ ]:
model = keras.models.load_model(model_path)
test_loss, test_accuracy = model.evaluate(test_ds, verbose=3)

y_score = model.predict(test_ds)
y_true = [int(y) for x, y in test_ds.unbatch()]
evaluate(model, history, test_accuracy, y_score, y_true, LOG_DIR, key, dataset)